In [ ]:
#!wget https://raw.githubusercontent.com/duyet/truyenkieu-word2vec/master/truyen_kieu_data.txt


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import urllib.request

url = "https://raw.githubusercontent.com/duyet/truyenkieu-word2vec/master/truyen_kieu_data.txt"
urllib.request.urlretrieve(url, "truyen_kieu_data.txt")


('truyen_kieu_data.txt', <http.client.HTTPMessage at 0x1b0a99eaa40>)

In [7]:
import re

In [8]:
with open('truyen_kieu_data.txt', 'r', encoding='utf-8') as f:
  lines = [line.strip() for line in f]
  clean_lines = [re.sub(r"^[\d\.,\s]+|,$|[,.!?:'\"”]|\n$", "", line) for line in lines]


In [9]:
def write_to_file(file_name, data):
  with open(file_name, 'w', encoding='utf-8') as f:
    i = 0
    for line in data:
      i += 1
      if i != len(data):
        f.write(line + "\n")
      else:
        f.write(line)

In [10]:
write_to_file('truyen_kieu_data_clean.txt', clean_lines)

In [11]:
test_size = 0.2
split_idx = int(test_size * len(lines))
test_data = clean_lines[:split_idx]
train_data = clean_lines[split_idx:]

In [12]:
write_to_file('truyen_kieu_train.txt', train_data)
write_to_file('truyen_kieu_test.txt', test_data)

In [30]:
from collections import defaultdict


def train_bigram(train_file, model_file):
    """Train bigram language model and save to model file
    """
    counts = defaultdict(int)  # count the n-gram
    context_counts = defaultdict(int)   # count the context
    with open(train_file, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')
            words.insert(0, '<s>')

            for i in range(1, len(words)):  # Note: starting at 1, after <s>
                counts[words[i-1] + ' ' +  words[i]] += 1 # Add bigram and bigram context
                context_counts[words[i-1]] += 1
                counts[words[i]] += 1    # Add unigram and unigram context
                context_counts[""] += 1

                pass

    # Save probabilities to the model file
    with open(model_file, 'w', encoding="utf-8") as fo:
        for ngram, count in counts.items():
            words = ngram.split(' ')
            words.pop()
            context = ' '.join(words)
            probability = counts[ngram]/context_counts[context]
            fo.write('%s\t%f\n' % (ngram, probability))
            pass

In [31]:
train_bigram('truyen_kieu_train.txt', 'bigram_model.txt')

In [35]:
def load_bigram_model(model_file):
    """Load the model file

    Args:
        model_file (str): Path to the model file

    Returns:
        probs (dict): Dictionary object that map from ngrams to their probabilities
    """
    probs = {}
    with open(model_file, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            ngram, p = line.split('\t')
            probs[ngram] = float(p)
            pass
    return probs

In [33]:
import random
def random_choice(distribution):
    r = random.random()      # số ngẫu nhiên trong [0, 1)
    cumulative = 0.0
    sentence = []

    for item, prob in distribution.items():
      if len(item) == 2 and item[0] == "<s>":
        if r <= cumulative:
          cumulative += prob
          sentence.append(item)
          break



In [36]:
model = load_bigram_model("bigram_model.txt")

print(sampling_bigram_model(model, 10))


NameError: name 'sampling_bigram_model' is not defined